# 술 추천 알고리즘

도수 유사도는 `유클리디안 거리`로 구했고, 풍미 및 기타 정보의 유사도는 `코사인유사도`로 구했다.  
술의 고유 id를 받아 비슷한 술의 고유 id를 return한다.

`코사인유사도`: 문서비교에 유용한 알고리즘으로 포도품종, 주원료 컬럼에 단어별 빈도를 파악해야하기때문

### general_recommendation():
도수 유사도 50% 풍미(4가지) 유사도 50% 고려해 주종 상관 없이 비슷한 술을 찾아줌

### wine_recommendation():
도수 유사도 50% 기타(중분류, 생산국가, 생산자/제조사, 포도품종, 맛) 50% 고려해 같은 주종 내에서 비슷한 와인을 찾아줌

### kor_recommendation():
도수 유사도 50% 기타(중분류, 생산자/제조사, 주원료, 맛) 50% 고려해 같은 주종 내에서 비슷한 전통주를 찾아줌

### the used variables of each alcohols

- 와인: 주종, 중분류, 주류명, 생산국가, 생산자/제조사, 포도품종, 용량, 도수, 음용온도, 추천음식, 맛
- 전통주 : 주종, 중분류, 주류명, 생산국가, 생산자/제조사, 주원료(막걸리만), 용량, 도수,음용온도, 맛
- 브랜디 : '주종','주류명','생산국가','용량','도수','특징'
- 위스키 : '주종','주류명','생산국가','용량','도수','특징'
- 국내주류 : '주종','주류명','생산국가','생산자/제조사','용량','도수','특징'

In [13]:
import numpy as np
import pandas as pd 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial import distance_matrix
from scipy.spatial.distance import squareform
from sklearn.preprocessing import MinMaxScaler
from string import punctuation
import re

In [26]:
alco = pd.read_csv('alcohol_df.csv')
alco

,id,주종,중분류,주류명,생산국가,생산자/제조사,포도품종,주원료,용량,도수,음용온도,추천음식,특징,당도,바디,산도,타닌
0,1,와인,레드,"7 컬러즈, '그랑 리제르바' 카베르네 소비뇽 뮈스카",칠레,세븐 컬러즈 7 Colores,"카베르네 소비뇽 (Cabernet Sauvignon) 95%, 뮈스까 (Muscat...",NaN,750,13.5,16~18 ℃,"채끝 스테이크, 크림 파스타, 블루치즈 버거 등과 잘 어울린다.",NaN,1.0,3.0,3.0,3.0
1,2,와인,레드,"GCF, 꼬뜨 드 뵈프",프랑스,GCF 그룹 GCF Group,"시라/쉬라즈 (Syrah/Shiraz) , 마르셀란 (Marselan)",NaN,750,13.0,16~18 ℃,"뵈프 부르기뇽, 안심 찹 스테이크, 로스트 비프 스테이크 등과 잘 어울린다.",NaN,1.0,3.0,3.0,2.0
2,3,와인,화이트,"가트, 리슬링",호주,가트 와인즈 Gatt Wines,리슬링 (Riesling) 100%,NaN,750,11.5,10~12 ℃,"샐러드, 해산물, 치즈 등과 잘 어울린다.",NaN,2.0,4.0,3.0,1.0
3,4,와인,레드,갈로 리빙스톤 콩코드,미국,갤로 패밀리 빈야드 Gallo Family Vineyard - E&J Gallo W...,콩코드 (Concord),NaN,750,11.0,14~16 ℃,"소시지, 육포, 떡볶이, 순대, 튀김, 족발, 만두, 자장면 등과 잘 어울린다.",NaN,1.0,4.0,3.0,3.0
4,5,와인,주정강화,글로리아 토니 포트 와인,포르투갈,빈센테 파리아 비뉴스 Vincente Faria VInhos,"틴타 로리즈 (Tinta Roriz) , 투리가 프란카 (Touriga Franca...",NaN,750,20.0,14~16 ℃,"리코타, 크림치즈, 말린과일, 케이크, 초콜렛 등 디저트와 잘 어울린다.",NaN,4.0,2.0,4.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1483,1484,과실주,NaN,톡소다,대한민국,무학,NaN,NaN,330,5.0,NaN,NaN,스페인산 화이트와인이 첨가된 스파클링 와인. 톡쏘는 탄산으로 더 청량하게,NaN,NaN,NaN,NaN
1484,1485,과실주,NaN,복분자주,대한민국,보해양조,NaN,NaN,360,15.0,NaN,NaN,보해 복분자주는 2004년 출시 이후 보해를 넘어 대한민국을 대표하는 과실주로 국민...,NaN,NaN,NaN,NaN
1485,1486,과실주,NaN,복분자주,대한민국,보해양죠,NaN,NaN,375,15.0,NaN,NaN,보해 복분자주는 2004년 출시 이후 보해를 넘어 대한민국을 대표하는 과실주로 국민...,NaN,NaN,NaN,NaN
1486,1487,과실주,NaN,매취순,대한민국,보해양조,NaN,NaN,750,14.0,NaN,NaN,‘매취순’은 국내 최대 매실농원인 전남 해남 ‘보해매실농원’에서 직접 수확한 국산 ...,NaN,NaN,NaN,NaN


In [27]:
# 도수 유사도 구하기
percent = alco['도수']
dist_pair = []

# y축에 임의로 0을 부여한 거리 순서쌍 생성
for i in range(0,len(percent)):
    temp = []
    temp.append(alco.loc[i]['도수'])
    temp.append(0)
    dist_pair.append(temp)

# get a distance matrix
df = pd.DataFrame(dist_pair, columns=['x', 'y'])
dist_matrix = distance_matrix(df.values, df.values)

# 정규화
min_max_scaler = MinMaxScaler()
regularised = min_max_scaler.fit_transform(dist_matrix)

# 1에서 빼줘서 더 가까운 것이 우선순위를 갖도록 변경하기
one_matrix = np.ones((1488,1488))

final_dist = one_matrix - regularised

# 확인
print(final_dist)

[[1.         0.99870801 0.99485199 ... 0.9961039  0.99870466 0.99348958]
 [0.99870634 1.         0.996139   ... 0.99480519 0.99740933 0.9921875 ]
 [0.99482536 0.99612403 1.         ... 0.99090909 0.99352332 0.98828125]
 ...
 [0.99611902 0.99483204 0.99099099 ... 1.         0.99740933 0.99739583]
 [0.99870634 0.99741602 0.99356499 ... 0.9974026  1.         0.99479167]
 [0.99353169 0.99224806 0.98841699 ... 0.9974026  0.99481865 1.        ]]


In [28]:
# flavour word list 만들기
flavour_list = [] # empty list

for i in range(0, len(alco)):
    temp = ""
    
    if alco.loc[i]['당도'] >= 2:
        temp = temp + "당도 "
    if alco.loc[i]['바디'] >= 2:
        temp = temp + "바디 "
    if alco.loc[i]['산도'] >= 2:
        temp = temp + "산도 "
    if alco.loc[i]['타닌'] >= 2:
        temp = temp + "타닌 "
    
    flavour_list.append(temp)

# 해당 리스트 데이터 프레임에 추가
alco["맛"] = pd.DataFrame({"flavour":flavour_list})
flavour = alco['맛']

# flavour 코사인 유사도 구하기
# instantiating and generating the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(flavour)

# generating the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)

# 확인
print(cosine_sim)

[[1.         1.         0.66666667 ... 0.         0.         0.        ]
 [1.         1.         0.66666667 ... 0.         0.         0.        ]
 [0.66666667 0.66666667 1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [29]:
# 도수, flavour를 모두 고려한 similarity 구하기 (weight는 각각 0.5)
new_sim = 0.5 * cosine_sim + 0.5 * final_dist

print(new_sim)

[[1.         0.99935401 0.83075933 ... 0.49805195 0.49935233 0.49674479]
 [0.99935317 1.         0.83140283 ... 0.4974026  0.49870466 0.49609375]
 [0.83074601 0.83139535 1.         ... 0.49545455 0.49676166 0.49414062]
 ...
 [0.49805951 0.49741602 0.4954955  ... 0.5        0.49870466 0.49869792]
 [0.49935317 0.49870801 0.4967825  ... 0.4987013  0.5        0.49739583]
 [0.49676585 0.49612403 0.49420849 ... 0.4987013  0.49740933 0.5       ]]


In [30]:
# 여러 주종 내 추천
# 주종에 상관 없이 도수와 풍미만 고려해 비슷한 술을 추천해줌
# 항목: 도수, 풍미 (약 6개 항목)

# 고유 id를 넣으면 해당 술과 비슷한 Top 10의 id를 return
def general_recommendation(input_id, new_sim = new_sim):
    
    # 주류명으로 index 찾기 
    idx = alco.index[alco['주류명'] == input_id].tolist() # Int64Index 형식이라 list로 바꾸어줌
    
    # 해당 index의 유사도 리스트 sort in descending order
    score_series = pd.Series(new_sim[idx[0]]).sort_values(ascending = False)
    
    # 유사도 Top 10의 index 추출
    top_10_indexes = list(score_series.iloc[1:11].index)

    # 유사도 1인 항목이 하나 더 있어서 자기 자신이 포함되는 경우에는 자신을 뺀 Top 10의 index 재추출
    if top_10_indexes[0] == idx[0]:
        top_10_indexes = list(score_series.iloc[1:12].index)
        top_10_indexes.remove(idx[0])
    
    # 고유 id를 담기 위한 empty list 생성
    top_10_id = []
    
    # id list
    for i in top_10_indexes:
        id = alco.loc[i]['주류명']
        top_10_id.append(id)
    
    return top_10_id

# test
print(general_recommendation('가트, 리슬링'))

['꽃 와인', '고도리 프룬와인', '뱅꼬레 로제와인', '도깨비술 11', '뱅꼬레 화이트와인', '노비볼레 로마냐 스푸만테 비앙코', '한스오차드 사과와인', '킬리빙빙, 스윗 립스', '바론 데 발스, 화이트', '더 빅 레드 몬스터']


In [31]:
# wine word list 만들기
wine_list = [] # empty list

# wine index list 추출
wine_idx = alco.index[alco['주류'] == "와인"].tolist()

# wine: category, percent, origin, producer, wine_grape, flavour
# 중분류, 도수, 생산자, 생산국가, 주요품종 

for i in wine_idx:
    temp = ""
    temp = temp + alco.loc[i]['중분류'] + " " + alco.loc[i]['생산국가'] + " " + alco.loc[i]['생산자/제조사'].replace(" ", "") + " " + alco.loc[i]['포도품종'].replace(" ", "") + " "
    
    for flavour in alco.loc[i]['맛']:
        temp = temp + flavour

    wine_list.append(temp)

# wine 코사인 유사도 구하기
# instantiating and generating the count matrix
count = CountVectorizer()
wine_matrix = count.fit_transform(wine_list)

# generating the cosine similarity matrix
wine_sim = cosine_similarity(wine_matrix, wine_matrix)

# wine 도수 유사도 구하기
wine_pair = []

# y축을 임의로 0을 부여한 거리 순서쌍 생성
for i in wine_idx:
    temp = []
    temp.append(alco.loc[i]['도수'])
    temp.append(0)
    wine_pair.append(temp)

# get a distance matrix
wine_df = pd.DataFrame(wine_pair, columns=['x', 'y'])
wine_matrix = distance_matrix(wine_df.values, wine_df.values)

# 정규화
min_max_scaler = MinMaxScaler()
wine_regularised = min_max_scaler.fit_transform(wine_matrix)

# 1에서 빼줘서 더 가까운 것이 우선순위를 갖도록 변경하기
wine_one_matrix = np.ones((len(wine_idx),len(wine_idx)))

wine_final_dist = wine_one_matrix - wine_regularised

# 도수 유사도, 타 정보 유사도 각각 0.5씩 weight 부여 후 새로운 matrix 생성
wine_new_sim = 0.5 * wine_sim + 0.5 * wine_final_dist

# 확인
print(wine_new_sim)

[[1.         0.64907766 0.48285672 ... 0.42265686 0.53654575 0.39908555]
 [0.65026813 1.         0.50798975 ... 0.50674002 0.61785113 0.42297553]
 [0.50526569 0.52122504 1.         ... 0.5402636  0.56108276 0.76233882]
 ...
 [0.44249813 0.51785113 0.53862962 ... 1.         0.65       0.55083589]
 [0.53773622 0.61785113 0.54784747 ... 0.63888889 1.         0.5227244 ]
 [0.41412315 0.42955447 0.75614686 ... 0.5493739  0.52930335 1.        ]]


In [32]:
# 전통주 word list 만들기
kor_list = [] # empty list

# 전통주 index list 추출
kor_idx = alco.index[alco['주류'] == "전통주"].tolist()

# 도수, 중분류, 주원료 , 생산자/제조사

for i in kor_idx:
    temp = ""
    temp = temp + str(alco.loc[i]['중분류']) + " " + str(alco.loc[i]['주원료'])+ " " +alco.loc[i]['생산자/제조사'].replace(" ", "")
    
    for flavour in alco.loc[i]['맛']:
        temp = temp + flavour

    kor_list.append(temp)

# 전통주 코사인 유사도 구하기
# instantiating and generating the count matrix
count = CountVectorizer()
kor_matrix = count.fit_transform(kor_list)

# generating the cosine similarity matrix
kor_sim = cosine_similarity(kor_matrix, kor_matrix)

# 전통주 도수 유사도 구하기
kor_pair = []

# y축을 임의로 0을 부여한 거리 순서쌍 생성
for i in kor_idx:
    temp = []
    temp.append(alco.loc[i]['도수'])
    temp.append(0)
    kor_pair.append(temp)

# get a distance matrix
kor_df = pd.DataFrame(kor_pair, columns=['x', 'y'])
kor_matrix = distance_matrix(kor_df.values, kor_df.values)

# 정규화
min_max_scaler = MinMaxScaler()
kor_regularised = min_max_scaler.fit_transform(kor_matrix)

# 1에서 빼줘서 더 가까운 것이 우선순위를 갖도록 변경하기
kor_one_matrix = np.ones((len(kor_idx),len(kor_idx)))

kor_final_dist = kor_one_matrix - kor_regularised

# 도수 유사도, 타 정보 유사도 각각 0.5씩 weight 부여 후 새로운 matrix 생성
kor_new_sim = 0.5 * kor_sim + 0.5 * kor_final_dist

# 확인
print(kor_new_sim)

[[1.         0.17613615 0.83000283 ... 0.47058824 0.42857143 0.44186047]
 [0.36247874 1.         0.33843537 ... 0.24509804 0.29761905 0.29069767]
 [0.82979024 0.13888889 1.         ... 0.48039216 0.41666667 0.43023256]
 ...
 [0.46875    0.01851852 0.47959184 ... 1.         0.89285714 0.80697674]
 [0.4375     0.18518519 0.42857143 ... 0.91176471 1.         0.88837209]
 [0.44791667 0.16666667 0.43877551 ... 0.82156863 0.88809524 1.        ]]


In [33]:
# 해당 id의 술이 와인인지 체크
def is_wine(input_id):
    temp_idx = alco.index[alco['주류명'] == input_id].tolist() # Int64Index 형식이라 list로 바꾸어줌
    result = alco.loc[temp_idx[0]]['주류'] == "와인"
    return result

# 와인의 고유 id를 넣으면 해당 와인과 비슷한 Top 10 와인의 id를 return
def wine_recommendation(input_id, wine_new_sim = wine_new_sim):
    
    # 주류명으로 index 찾기 
    idx = alco.index[alco['주류명'] == input_id].tolist() # Int64Index 형식이라 list로 바꾸어줌
    
    # wine_idx list 내에서 몇번째 와인인지 구하기
    w_idx = wine_idx.index(idx[0])
    
    # 해당 index의 유사도 리스트 sort in descending order
    score_series = pd.Series(wine_new_sim[w_idx]).sort_values(ascending = False)
    
    # 유사도 Top 10의 index 추출
    wine_top_10_indexes = list(score_series.iloc[1:11].index)

    # 유사도 1인 항목이 하나 더 있어서 자기 자신이 포함되는 경우에는 자신을 뺀 Top 10의 index 재추출
    if wine_top_10_indexes[0] == w_idx:
        wine_top_10_indexes = list(score_series.iloc[1:12].index)
        wine_top_10_indexes.remove(w_idx)
    
    # 고유 id를 담기 위한 empty list 생성
    wine_top_10_id = []
    
    # id list
    for i in wine_top_10_indexes:
        index = wine_idx[i] # wine list에서 몇번째인지가 아니라 전체 술 list에서 몇번째인지 구함
        id = alco.loc[index]['주류명']
        wine_top_10_id.append(id)
    
    return wine_top_10_id

In [34]:
# 해당 id의 술이 전통주인지 체크
def is_kor(input_id):
    temp_idx = alco.index[alco['주류명'] == input_id].tolist() # Int64Index 형식이라 list로 바꾸어줌
    result = alco.loc[temp_idx[0]]['주류'] == "전통주"
    return result

#  전통주의 고유 id를 넣으면 해당 전통주와 비슷한 Top 10 전통주의 주류명를 return
def kor_recommendation(input_id, kor_new_sim = kor_new_sim):
    
    # 주류명으로 index 찾기 
    idx = alco.index[alco['주류명'] == input_id].tolist() # Int64Index 형식이라 list로 바꾸어줌
    
    # kor_idx list 내에서 몇번째 전통주인지 구하기
    k_idx = kor_idx.index(idx[0])
    
    # 해당 index의 유사도 리스트 sort in descending order
    score_series = pd.Series(kor_new_sim[k_idx]).sort_values(ascending = False)
    
    # 유사도 Top 10의 index 추출
    kor_top_10_indexes = list(score_series.iloc[1:11].index)

    # 유사도 1인 항목이 하나 더 있어서 자기 자신이 포함되는 경우에는 자신을 뺀 Top 10의 index 재추출
    if kor_top_10_indexes[0] == k_idx:
        kor_top_10_indexes = list(score_series.iloc[1:12].index)
        kor_top_10_indexes.remove(k_idx)
    
    # 고유 id를 담기 위한 empty list 생성
    kor_top_10_id = []
    
    # id list
    for i in kor_top_10_indexes:
        index = kor_idx[i] # wine list에서 몇번째인지가 아니라 전체 술 list에서 몇번째인지 구함
        id = alco.loc[index]['주류명']
        kor_top_10_id.append(id)
    
    return kor_top_10_id

In [37]:
# 테스트로 임의의 주류명 넣음
test_num = '서울의 밤'

# 주류명 출력  

print(general_recommendation(test_num)) # 고유 id를 넣으면 주류상관없이 비슷한 술 top10 추천  

print(is_wine(test_num)) # 해당 id의 술이 wine인지 체크 

print(is_kor(test_num))# 전통주인지 체크

if is_wine(test_num) == True:
    print(wine_recommendation(test_num))# 와인의 고유 id를 넣으면 해당 와인과 비슷한 Top 10 와인의 주류명 출력 
elif is_kor(test_num) == True:
    print(kor_recommendation(test_num))# 전통주 id를 넣으면 해당 전통주와 비슷한 top10 와인의 주류명을 출력 
else :
    print('해당 주류는 와인,전통주가 아닙니다.')

['달홀진주25', '서울의 밤', '연천 우주', '톡 한잔 소주', '경주교동법주', '메종 카스텔, 세리 리미떼 꽁드리유', '메종 카스텔, 푸이 퓌메', '펠릭스 솔리스, 아날리비아', '7 컬러즈, 브륏 샤르마', '메종 카스텔, 퀴베 블랑쉬 브뤼']
False
True
['명랑 스컬', '여유25', '장수홍삼주', '호담 300', '뱁새', '화주', '달홀진주25', '빡치주', '산내울 복분자주', '산내울 사과애주']
